In [ ]:
# load required packages
import optuna
import numpy as np
import pandas as pd
import tensorflow as tf
from lightgbm import LGBMRegressor
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
def save_result(y_valid, y_pred, filepath):
    r2 = pd.DataFrame({'True Value': y_valid,'Prediction Value': y_pred})
    r2.to_csv(filepath,index=False)

# 1. Load data

In [ ]:
# load phenotype label
gw = pd.read_csv('gw.csv', index_col=0)
gl = pd.read_csv('gl.csv', index_col=0)

In [ ]:
# load gsctool features
dataA_features = pd.read_csv('dataA.gsc.csv',index_col=0)
dataB_features = pd.read_csv('dataB.gsc.csv',index_col=0)
dataC_features = pd.read_csv('dataC.gsc.csv',index_col=0)

In [ ]:
dataA_features.index.name = 'Run'
dataB_features.index.name='Run'
dataC_features.index.name = 'Sample Name'

In [ ]:
# merge featuers ann label
gw_dataA = pd.merge(gw.iloc[:, 1], dataA_features, on = 'Run').dropna()
gw_dataB = pd.merge(gw.iloc[:, 1], dataB_features, on = 'Run').dropna()
gw_dataC = pd.merge(gw.iloc[:, [0,1]], dataC_features, on = 'Sample Name').set_index('Sample Name').dropna()
gl_dataA = pd.merge(gl.iloc[:, 1], dataA_features, on = 'Run').dropna()
gl_dataB = pd.merge(gl.iloc[:, 1], dataB_features, on = 'Run').dropna()
gl_dataC = pd.merge(gl.iloc[:, [0,1]], dataC_features, on = 'Sample Name').set_index('Sample Name').dropna()

In [ ]:
gl_dataA_value = gl_dataA.iloc[:, 1:].values
gl_dataA_label = gl_dataA.iloc[:, 0].values
gl_dataB_value = gl_dataB.iloc[:, 1:].values
gl_dataB_label = gl_dataB.iloc[:, 0].values
gl_dataC_value = gl_dataC.iloc[:, 1:].values
gl_dataC_label = gl_dataC.iloc[:, 0].values

In [ ]:
gw_dataA_value = gw_dataA.iloc[:, 1:].values
gw_dataA_label = gw_dataA.iloc[:, 0].values
gw_dataB_value = gw_dataB.iloc[:, 1:].values
gw_dataB_label = gw_dataB.iloc[:, 0].values
gw_dataC_value = gw_dataC.iloc[:, 1:].values
gw_dataC_label = gw_dataC.iloc[:, 0].values

# 2 lightGBM + Optuna
The following example is grain length predictions of dataA, which can be replaced with other phenotypes of other data

In [ ]:
# split the training and testing data
def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = np.concatenate([X_train, X_part], 0)
            y_train = np.concatenate([y_train, y_part], 0)
    return X_train, y_train, X_valid, y_valid

In [ ]:
X_train, y_train, X_valid, y_valid = get_k_fold_data(5,1,gl_dataA_value,gl_dataA_label)

In [ ]:
# Optuna search the best hyperparameter 
def objective(trial):
    param = {
        'metric': 'mse', 
        "boosting_type": "gbdt",                
        "seed": 42,
        "verbosity": -1,
        "n_estimators": trial.suggest_int('n_estimators', 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
    }
    
    lgb=LGBMRegressor(**param)
    lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)])
    return lgb.score(X_valid, y_valid)

In [ ]:
# Search the best hyperparameter
optuna.logging.set_verbosity(optuna.logging.WARNING) # 压缩报告信息
study_tuner = optuna.create_study(direction='maximize')
study_tuner.optimize(objective, n_trials=100) 

In [ ]:
# exhibit the beat hyperparameter
trial = study_tuner.best_trial
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

In [ ]:
# laod the best hyperparameter
def objective(trial):
    param = {
        'metric': 'mse', 
        "boosting_type": "gbdt",                
        "seed": 42,
        "verbosity": -1,
        "n_estimators": trial.suggest_int('n_estimators', 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
    }
    
    lgb=LGBMRegressor(**param)
    model = lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)])
    y_pred = model.predict(X_valid)
    return lgb, y_pred

In [ ]:
# get the model with the best hyperparameter
lgb, y_pred = objective(study_tuner.best_trial)

In [ ]:
# save the predict result, which can be used for evaluate the performance of model
save_result(y_valid,y_pred, 'tmp.lgb.csv')

# 3 GRU + Optuna
The following example is a grain length prediction for dataA
1. Dataset can be replaced
2. GRU can be replaced by BiGRU

In [ ]:
# split the training and testing data
def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = np.concatenate([X_train, X_part], 0)
            y_train = np.concatenate([y_train, y_part], 0)
    return X_train.reshape(-1,1,X_train.shape[1]).astype(np.float32), y_train.astype(np.float32), X_valid.reshape(-1,1,X_train.shape[1]).astype(np.float32), y_valid.astype(np.float32)

In [ ]:
X_train, y_train, X_valid, y_valid = get_k_fold_data(5,1,gl_dataA_value,gl_dataA_label)

In [ ]:
def objective(trial):
    # Parameters
    L2 = trial.suggest_float("l", 1e-5, 1e-2, log=True)
    BATCH_SIZE = trial.suggest_int("batch_size", 16, 128, step=8)
    EPOCHS = trial.suggest_int("epochs", 10,100, step=10)
    LR = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    OPT = trial.suggest_categorical("optimizer", [Adam, SGD, RMSprop])
    
    model = Sequential()
    model.add(GRU(units=64, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(l=L2)))
    model.add(GRU(units=64, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(l=L2)))
    model.add(GRU(units=32))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1))

    model.compile(optimizer=OPT(learning_rate=LR), loss='mse', metrics=['mae'])

    H = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=EPOCHS, batch_size=BATCH_SIZE)
    y_pred = model.predict(X_valid).reshape(y_valid.shape[0],)
    return r2_score(y_valid,y_pred)

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

In [ ]:
trial = study.best_trial
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

In [ ]:
# laod the best hyperparameter
def objective(trial):
    # Parameters
    L2 = trial.suggest_float("l", 1e-5, 1e-2, log=True)
    BATCH_SIZE = trial.suggest_int("batch_size", 16, 128, step=8)
    EPOCHS = trial.suggest_int("epochs", 10,100, step=10)
    LR = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    OPT = trial.suggest_categorical("optimizer", [Adam, SGD, RMSprop])
    
    model = Sequential()
    model.add(GRU(units=128, return_sequences=True, kernel_regularizer = tf.keras.regularizers.l2(l=L2)))
    model.add(GRU(units=64,return_sequences=True, kernel_regularizer = tf.keras.regularizers.l2(l=L2)))
    model.add(GRU(units=32))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1))

    model.compile(optimizer=OPT(learning_rate=LR), loss='mse', metrics=['mae'])

    H = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=EPOCHS, batch_size = BATCH_SIZE)
    y_pred = model.predict(X_valid).reshape(y_valid.shape[0],)

    return H, y_pred

In [ ]:
# get the model with the best hyperparameter
model, y_pred = objective(study.best_trial)

In [ ]:
# save the predict result, which can be used for evaluate the performance of model
save_result(y_valid, y_pred, 'tmp.gru.csv')